# Custom DeepSearch with Flexible LLM and Search

This document guides you on how to rebuild the `DeepSearch` tool from the `vinagent` library to allow passing a custom LLM (e.g., OpenAI) and a search engine (e.g., Tavily, Google).

### Disadvantages of the original library version:
- **Hardcoded LLM**: Only uses `ChatTogether`.
- **Hardcoded Search**: Only uses `TavilyClient` with default parameters.
- **Rigid Structure**: Hard to customize the logic of nodes in the Graph.

### Enhancements in this version:
1. **Configurable LLM**: Pass any LangChain model into the constructor.
2. **Configurable Search**: Pass a custom search function.
3. **Modular Design**: Nodes are designed as methods that are easy to override.
4. **State Management**: Uses `Annotated` and `operator.add` to manage chapter/section lists automatically and cleanly.

In [1]:
%pip install vinagent==0.0.6.post3

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import re
import operator
from typing import TypedDict, Annotated, List, Optional, Callable
from pydantic import BaseModel
from dotenv import load_dotenv
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from tavily import TavilyClient

load_dotenv()

True

In [3]:
from datetime import datetime
from vinagent.register import primary_function

@primary_function
def get_current_time() -> str:
    """
    Get the current date and time. Use this to know 'today's' date.
    Returns:
        str: Current date and time in YYYY-MM-DD HH:MM:SS format.
    """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print("Custom time tool defined.")

Custom time tool defined.


## 1. State Definition

We use `Annotated` with `operator.add` so that results from nodes are automatically merged into the current list instead of overwritten.

In [4]:
class AgentState(TypedDict):
    task: str
    plan: List[str]
    draft: str
    critique: str
    adjustment: List[str]
    # Use reducer operator.add to merge lists instead of overwriting
    sections: Annotated[List[str], operator.add]
    chapters: Annotated[List[str], operator.add]
    revision_number: int
    max_revisions: int
    max_chapters: int
    max_paragraphs_per_chapter: int
    max_critical_queries: int
    number_of_chapters: int
    current_chapter_order: int

## 2. FlexibleDeepSearch Class

This class takes `llm` and `search_func` during initialization, making it compatible with any provider.

In [5]:
class FlexibleDeepSearch:
    PLAN_PROMPT = """You are an expert writer tasked with writing a high level outline of an analytical essay on the topic. \
    Write such an outline for the user provided topic. Give an outline of the essay along with any relevant notes \
    or instructions for the chapters. Not more than {max_chapters} chapters. The output should be in the following format:
    1. Chapter 1
    2. Chapter 2
    ..."""

    WRITER_PROMPT = """You are an researcher assistant tasked with writing excellent {max_paragraphs_per_chapter} paragraph research article.\
    Generate the best research possible for the chapter based on user's collected information. \
    If the user provides critique and suggested adjustment, respond with a revised version of your previous content. \
    The article should include comparisions, statistics data, and references to make clear the arguments. \
    Directly generate without any explanation. \
    Utilize all the information below as needed: \

    ------
    - Previous content:
    {content}
    - Critique:
    {critique}
    - Suggested Adjustment:
    {adjustment}
    """

    REFLECTION_PROMPT = """You are a teacher grading an research submission. \
    Generate critique and recommendations for the user's submission. \
    Provide detailed recommendations, including requests for coherence & cohension, lexical resource, task achievement, comparison, statistics data. \
    Only generate critique and recommendations less than 200 words."""

    RESEARCH_CRITIQUE_PROMPT = """
    You are a researcher charged with critiquing information as outlined below. \
    Generate a list of search queries that will gather any relevant information. Only generate maximum {max_critical_queries} queries.
    """

    def __init__(self, llm, search_func: Callable[[str], List[str]]):
        self.llm = llm
        self.search_func = search_func
        self.graph = self._build_graph()

    def _build_graph(self):
        builder = StateGraph(AgentState)
        builder.add_node("planner", self.plan_node)
        builder.add_node("generate", self.generation_node)
        builder.add_node("reflect", self.reflection_node)
        builder.add_node("research_critique", self.research_critique_node)
        
        builder.add_edge(START, "planner")
        builder.add_edge("planner", "generate")
        builder.add_conditional_edges(
            "generate", self.should_continue, {END: END, "reflect": "reflect"}
        )
        builder.add_edge("reflect", "research_critique")
        builder.add_edge("research_critique", "generate")
        
        return builder.compile(checkpointer=MemorySaver())

    def plan_node(self, state: AgentState):
        print(f"[Planner] Planning chapters for: {state['task'][:50]}...")
        messages = [
            SystemMessage(content=self.PLAN_PROMPT.format(max_chapters=state['max_chapters'])),
            HumanMessage(content=state["task"]),
        ]
        response = self.llm.invoke(messages)
        
        list_tasks = [t for t in response.content.split("\n") if re.match(r"^\d+\. ", t.strip())]
        return {
            "plan": list_tasks,
            "current_chapter_order": 0,
            "number_of_chapters": len(list_tasks),
        }

    def generation_node(self, state: AgentState):
        idx = state["current_chapter_order"]
        chapter_outline = state["plan"][idx]
        chapter_title = chapter_outline.split("\n")[0].strip()
        
        print(f"[Generator] Writing chapter: {chapter_title}")
        
        # Perform search
        search_results = self.search_func(chapter_outline)
        
        # If there is information from the previous research_critique step
        if state.get("adjustment"):
            search_results.extend(state["adjustment"])

        messages = [
            SystemMessage(content=self.WRITER_PROMPT.format(
                max_paragraphs_per_chapter=state["max_paragraphs_per_chapter"],
                content="\n".join(search_results),
                critique=state.get("critique", ""),
                adjustment="\n".join(state.get("adjustment", [])),
            )),
            HumanMessage(content=f"Outline: {chapter_outline}")
        ]
        
        response = self.llm.invoke(messages)
        
        # Update revision_number and chapter_order
        is_last_revision = state.get("revision_number", 1) >= state["max_revisions"]
        
        new_data = {
            "draft": response.content,
            "sections": [chapter_title]
        }
        
        if is_last_revision:
            new_data["chapters"] = [f"## {chapter_title}\n{response.content}"]
            new_data["current_chapter_order"] = idx + 1
            new_data["revision_number"] = 1
        else:
            new_data["revision_number"] = state["revision_number"] + 1
            
        return new_data

    def reflection_node(self, state: AgentState):
        print("[Reflector] Reflecting on draft...")
        messages = [
            SystemMessage(content=self.REFLECTION_PROMPT),
            HumanMessage(content=state["draft"]),
        ]
        response = self.llm.invoke(messages)
        return {"critique": response.content}

    def research_critique_node(self, state: AgentState):
        print("[Researcher] Refining info based on critique...")
        messages = [
            SystemMessage(content=self.RESEARCH_CRITIQUE_PROMPT.format(
                max_critical_queries=state["max_critical_queries"]
            )),
            HumanMessage(content=f"Critique to address: {state['critique']}"),
        ]
        response = self.llm.invoke(messages)
        
        queries = [q for q in response.content.split("\n") if re.match(r"^\d+\. ", q.strip())]
        new_results = []
        for q in queries:
            new_results.extend(self.search_func(q))
            
        return {"adjustment": new_results}

    def should_continue(self, state: AgentState):
        if state["current_chapter_order"] >= state["number_of_chapters"]:
            return END
        return "reflect"

    def run(self, query: str, config: dict = {}):
        initial_state = {
            "task": query,
            "max_chapters": config.get("max_chapters", 3),
            "max_paragraphs_per_chapter": config.get("max_paragraphs_per_chapter", 3),
            "max_critical_queries": config.get("max_critical_queries", 3),
            "max_revisions": config.get("max_revisions", 1),
            "revision_number": 1,
            "sections": [],
            "chapters": [],
            "adjustment": []
        }
        
        # Run the graph
        final_state = self.graph.invoke(initial_state, {"configurable": {"thread_id": "custom_deepsearch"}})
        
        content = f"# I. Planning\n" + "\n".join(final_state["sections"]) + "\n\n# II. Results\n" + "\n\n".join(final_state["chapters"])
        return content

## 3. Create Search Tool with Tavily

This function will be passed into `FlexibleDeepSearch`.

In [6]:
tavily = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

def tavily_search(query: str) -> List[str]:
    response = tavily.search(query=query, max_results=3)
    return [r["content"] for r in response["results"]]

## 4. Demo with OpenAI

Now we can easily pass `ChatOpenAI` into the class.

In [ ]:
openai_llm = ChatOpenAI(model="gpt-4o-mini")

deep_search_openai = FlexibleDeepSearch(llm=openai_llm, search_func=tavily_search)

result = deep_search_openai.run(
    "Analysis of electric vehicle market potential in Vietnam in 2026",
    config={"max_chapters": 2, "max_revisions": 1}
)

from IPython.display import Markdown
Markdown(result)

[Planner] Planning chapters for: Analysis of electric vehicle market potential in V...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[Generator] Writing chapter: 1. Chapter 1: Current Landscape and Driving Forces


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[Reflector] Reflecting on draft...


...

[Researcher] Refining info based on critique...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[Generator] Writing chapter: 2. Chapter 2: Future Projections and Challenges


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# I. Planning
1. Chapter 1: Current Landscape and Driving Forces
2. Chapter 2: Future Projections and Challenges

# II. Results
## 1. Chapter 1: Current Landscape and Driving Forces
Chapter 1: Current Landscape and Driving Forces

The intricate relationship between landscape elements and land uses is significantly influenced by various driving forces, fueling consistent landscape change. These forces encompass a wide range of factors, including economic, social, environmental, and technological influences, which interact dynamically to reshape landscapes over time. For instance, a study by Forman and Alexander (1998) illustrates how urban sprawl, driven by population growth and economic development, leads to significant alterations in land use patterns and ecosystem services. In the period from 2000 to 2010, it was reported that urban areas in the United States expanded by approximately 30 percent, showcasing how driving forces directly correlate with alterations in land use (U.S. Census Bureau, 2010).

Despite the inherent dynamism of landscapes, where change is a persistent feature, it is critical to recognize that the underlying driving forces are evolving, necessitating new approaches to understanding their impacts. Recent advancements in remote sensing and geographic information systems (GIS) enable researchers to capture these dynamics more accurately, providing deeper insights into landscape changes. For example, a study conducted by Turner et al. (2015) employed GIS technology to analyze land-use changes in the Brazilian Amazon, revealing a direct linkage between agricultural expansion and deforestation rates. This technological progression not only facilitates the identification of driving forces but also aids in forecasting future landscape changes in response to global trends such as climate change and urbanization.

In summarizing the current state of the art in landscape change research, it is clear that a comprehensive understanding of driving forces is essential for effective landscape management and planning. Innovative conceptual frameworks and methodological approaches are needed to capture the complex interplay of these forces. Recent efforts, such as those by Sudhira et al. (2004), have emphasized the integration of socio-economic factors with ecological modeling to design more sustainable landscape solutions. By addressing the multifaceted nature of driving forces, researchers can provide valuable insights into mitigating adverse effects and enhancing landscape resilience, leading to sustainable land use practices that account for both human and ecological needs.

## 2. Chapter 2: Future Projections and Challenges
**Chapter 2: Future Projections and Challenges**

**Introduction**  
In contemporary society, understanding the future is crucial in addressing the imminent challenges we face across various sectors. With the advent of new technologies and changes in demographics, it is essential to acknowledge the multifaceted nature of these emerging challenges. This chapter aims to bring together qualitative approaches to future projections, emphasizing the importance of considering multiple possible futures to better inform policy and strategic planning. 

**2.1 Demographic Shifts and Their Implications**  
The demographic landscape of the United States is undergoing drastic transformations, accelerated by immigration policies such as the 1965 Immigration and Nationality Act. Foreign-born Americans and their descendants represent a significant driver of U.S. population growth, with projections indicating an increase from 45 million to 78 million immigrants by 2055, marking a 74% growth compared to 30% growth in the U.S.-born population (Pew Research Center, 2023). Notably, Hispanic and Asian populations are expected to continue significantly influencing demographic composition, with Asians potentially surpassing Hispanics as the largest foreign-born group by 2055. The implications of this exponential growth extend to labor markets, social services, and cultural dynamics, necessitating comprehensive strategies to accommodate a more diverse society.

**2.2 Technological Advancement and Its Challenges**  
Technological innovation is also a vital factor influencing future projections. The pace of technological change is unprecedented, reshaping industries and altering job landscapes. A comparative analysis of historical and contemporary approaches reveals that the integration of artificial intelligence and automation presents both opportunities and challenges, including workforce displacement (Brynjolfsson & McAfee, 2014). For instance, while automation can increase productivity, it often leads to significant job losses in traditional sectors. Addressing these challenges requires a forward-thinking approach, emphasizing the need for education systems to adapt and prepare a workforce skilled in emerging technologies.

**2.3 Environmental Implications and Sustainability**  
Moreover, environmental sustainability emerges as a pressing challenge tied to future projections. Rapid population growth and urbanization, particularly from immigrant populations, heighten demands on natural resources and contribute to environmental degradation. According to the UN, global urban areas could house 68% of the world's population by 2050, intensifying the need for sustainable urban planning (United Nations Department of Economic and Social Affairs, 2018). The intersection of demographics, technology, and environmental sustainability requires an integrative approach to policy-making that considers long-term impacts. Engaging diverse stakeholders through qualitative methodologies, such as scenario planning, can ensure that policymakers effectively navigate the complexities of future challenges while striving for equitable outcomes.

**Conclusion**  
In summary, the future is shaped by an intricate interplay of demographic changes, technological advancements, and environmental concerns. To successfully address these challenges, it is vital to employ a multidimensional perspective that embraces diverse futures and their implications. As we progress through the 21st century, enhancing our understanding of these interconnected issues will be pivotal for sustainable development and social cohesion. 

**References**  
Brynjolfsson, E., & McAfee, A. (2014). The Second Machine Age: Work, Progress, and Prosperity in a Time of Brilliant Technologies. W. W. Norton & Company.  
Pew Research Center. (2023). U.S. Immigrant Population Projections.  
United Nations Department of Economic and Social Affairs. (2018). World Urbanization Prospects: The 2018 Revision. 